In [1]:
import autograd.numpy as np
import autograd.numpy.random as npr
from autograd.scipy.misc import logsumexp
from autograd import grad, jacobian

from torchhmm.torchhmm import HMMNormalizer

import torch
from torch.autograd import Function, Variable

In [2]:
def normalizer(log_pi0, log_As, log_likes):
    T, K = log_likes.shape
    assert log_pi0.shape == (K,)
    assert log_As.shape == (T-1, K, K)
    
    alpha = log_pi0 + log_likes[0]
    for t in range(T-1):
        alpha = logsumexp(alpha + log_As[t].T, axis=1)
        alpha += log_likes[t+1]
    
    return logsumexp(alpha)

In [3]:
T = 10
K = 3
log_pi0 = npr.randn(K).astype(np.float32)
log_As = npr.randn(T-1, K, K).astype(np.float32)
log_likes = npr.randn(T, K).astype(np.float32)

In [4]:
# Use autograd to get the gradient of the normalizer wrt the inputs
Z = normalizer(log_pi0, log_As, log_likes)
d_log_pi0 = grad(normalizer, 0)(log_pi0, log_As, log_likes)
d_log_As = grad(normalizer, 1)(log_pi0, log_As, log_likes)
d_log_likes = grad(normalizer, 2)(log_pi0, log_As, log_likes)

In [5]:
# Compare against the torch implementation
v_log_pi0 = Variable(torch.from_numpy(log_pi0), requires_grad=True)
v_log_As = Variable(torch.from_numpy(log_As), requires_grad=True)
v_log_likes = Variable(torch.from_numpy(log_likes), requires_grad=True)
Z2 = HMMNormalizer.apply(v_log_pi0, v_log_As, v_log_likes)

loss = Z2
loss.backward()
assert np.allclose(Z, Z2.data.numpy())

In [6]:
assert np.allclose(d_log_pi0, v_log_pi0.grad.data.numpy())
assert np.allclose(d_log_As, v_log_As.grad.data.numpy())
assert np.allclose(d_log_likes, v_log_likes.grad.data.numpy())